In [1]:
pwd

'D:\\DCU\\Practicum\\Datasets\\DeepFashion2\\Mask RCNN\\Clothes\\IPNYBs'

In [2]:
%cd ..

D:\DCU\Practicum\Datasets\DeepFashion2\Mask RCNN\Clothes


In [3]:
cd main8/Mask_RCNN

D:\DCU\Practicum\Datasets\DeepFashion2\Mask RCNN\Clothes\main8\Mask_RCNN


In [4]:
ls

 Volume in drive D is Study
 Volume Serial Number is 16C4-DB1E

 Directory of D:\DCU\Practicum\Datasets\DeepFashion2\Mask RCNN\Clothes\main8\Mask_RCNN

10-06-2020  19:02    <DIR>          .
10-06-2020  19:02    <DIR>          ..
31-03-2019  23:03               569 .gitignore
10-06-2020  19:02    <DIR>          __pycache__
08-06-2020  13:54            15,849 clothes.py
02-06-2020  14:49         1,417,321 Fashion_MaskRCNN_V1.ipynb
03-06-2020  16:24         1,414,007 Fashion_MaskRCNN_V2.ipynb
31-03-2019  23:03             1,095 LICENSE
31-03-2019  23:03                58 MANIFEST.in
10-06-2020  10:49    <DIR>          mrcnn
06-06-2020  17:46            15,209 newclothes.py
31-03-2019  23:03            13,771 README.md
31-03-2019  23:03               119 requirements.txt
31-03-2019  23:03                99 setup.cfg
31-03-2019  23:03             2,518 setup.py
              11 File(s)      2,880,615 bytes
               4 Dir(s)  412,324,196,352 bytes free


In [7]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Root directory of the project
ROOT_DIR = "D:\DCU\Practicum\Datasets\DeepFashion2\Mask RCNN\Clothes\main8\Mask_RCNN"

MODEL_DIR = "D:\DCU\Practicum\Datasets\DeepFashion2\Mask RCNN\Clothes\weights\10th_june\mask_rcnn_object_0100.h5"
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

import clothes

%matplotlib inline

Using TensorFlow backend.


In [8]:
config = clothes.CustomConfig()

In [9]:
# Override the training configurations with a few
# changes for inferencing.
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                26
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [10]:
# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"

In [11]:

def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [13]:
CUSTOM_DIR = r"D:\DCU\Practicum\Datasets\DeepFashion2\Mask RCNN\Clothes\main8\dataset"

In [14]:
# Load validation dataset
dataset = clothes.CustomDataset()
dataset.load_custom(CUSTOM_DIR, "val")

objects: ['short sleeve dress']
numids [10]
objects: ['long sleeve top']
numids [2]
objects: ['vest dress']
numids [12]
objects: ['shorts']
numids [7]
objects: ['short sleeve dress']
numids [10]
objects: ['shorts']
numids [7]
objects: ['skirt', 'short sleeve top']
numids [9, 1]
objects: ['skirt', 'sling']
numids [9, 6]
objects: ['trousers', 'long sleeve top']
numids [8, 2]
objects: ['trousers', 'long sleeve top']
numids [8, 2]
objects: ['long sleeve outwear', 'shorts']
numids [4, 7]
objects: ['long sleeve outwear', 'trousers']
numids [4, 8]
objects: ['shorts', 'short sleeve top']
numids [7, 1]
objects: ['shorts', 'long sleeve top']
numids [7, 2]
objects: ['short sleeve top', 'shorts']
numids [1, 7]
objects: ['trousers', 'long sleeve outwear']
numids [8, 4]
objects: ['long sleeve dress']
numids [11]
objects: ['trousers', 'short sleeve dress']
numids [8, 10]
objects: ['vest dress']
numids [12]
objects: ['shorts', 'long sleeve top']
numids [7, 2]
objects: ['short sleeve top', 'vest dress'

In [15]:
# Must call before using the dataset
dataset.prepare()

print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

Images: 50
Classes: ['BG', 'short sleeve top', 'long sleeve top', 'short sleeve outwear', 'long sleeve outwear', 'vest', 'sling', 'shorts', 'trousers', 'skirt', 'short sleeve dress', 'long sleeve dress', 'vest dress', 'sling dress']


In [16]:
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.


In [18]:
# Set path to balloon weights file

# Download file from the Releases page and set its path
# https://github.com/matterport/Mask_RCNN/releases
# weights_path = "/path/to/mask_rcnn_idv.h5"

# Or, load the last model you trained
weights_path = r"D:\DCU\Practicum\Datasets\DeepFashion2\Mask RCNN\Clothes\weights\10th_june\mask_rcnn_object_0100.h5"

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

Loading weights  D:\DCU\Practicum\Datasets\DeepFashion2\Mask RCNN\Clothes\weights\10th_june\mask_rcnn_object_0100.h5


In [19]:
#list=[0,1,2,3,4,5,6]

In [ ]:
#for i in list:
image_id = 10
image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)
info = dataset.image_info[image_id]
print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id, 
                               dataset.image_reference(image_id)))

# Run object detection
results = model.detect([image], verbose=1)

# Display results
ax = get_ax(1)
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                    dataset.class_names, r['scores'], ax=ax,
                    title="Predictions")
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

image ID: object.051922.jpg (10) D:\DCU\Practicum\Datasets\DeepFashion2\Mask RCNN\Clothes\main8\dataset\val\051922.jpg
Processing 1 images
image                    shape: (1024, 1024, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 26)               min:    0.00000  max: 1024.00000  int32
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
